In [1]:
import ee
import pandas as pd
import numpy as np
import geopandas as gpd
import json
from shapely.geometry import Point
# import geemap.core as geemap
import folium
from IPython.display import Image
import geemap

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AQSTgQFGPXQOWlsHqZyUApD6hKYDkIJbY2JsrPzZlm1cpD_sEoLEKsOc3GY



Successfully saved authorization token.


In [4]:
studyArea = (
    ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
    .filter(ee.Filter.equals('ADM1_NAME', 'Taraba'))
)
studyArea.di

In [11]:
m = geemap.Map()
m.add_layer(studyArea, {}, 'Taraba', False)
m.center_object(studyArea, 7)
m

Map(center=[8.029871402180769, 10.82849788835878], controls=(WidgetControl(options=['position', 'transparent_b…

In [13]:
chirp = (
    ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD")
)

In [28]:
firsImage = ee.Date(ee.List(chirp.get('date_range')).get(0))
lastImage = ee.Date("2024-11-09")

In [18]:
# set resolution
resolution = 5550

In [19]:
# set time scale information for spi
timestep = '2'

In [26]:
# monthly spi
thresholdMonths = ee.Number(12)

# for lag of one for
timedif = lastImage.difference(firsImage, 'month').divide(ee.Number.parse(timestep))

list_ = ee.List.sequence(0, timedif)   #to create the list

In [31]:
# map the date

def timeList(month):
    zero = ee.Number(0)
    delta = zero.subtract(month).multiply(ee.Number.parse(timestep))
    latestDate = lastImage.advance(1, 'day')
    return latestDate.advance(delta, 'month')

timeListDate = list_.map(timeList)

In [35]:
print(timeListDate.getInfo())

[{'type': 'Date', 'value': 1731196800000}, {'type': 'Date', 'value': 1725926400000}, {'type': 'Date', 'value': 1720569600000}, {'type': 'Date', 'value': 1715299200000}, {'type': 'Date', 'value': 1710028800000}, {'type': 'Date', 'value': 1704844800000}, {'type': 'Date', 'value': 1699574400000}, {'type': 'Date', 'value': 1694304000000}, {'type': 'Date', 'value': 1688947200000}, {'type': 'Date', 'value': 1683676800000}, {'type': 'Date', 'value': 1678406400000}, {'type': 'Date', 'value': 1673308800000}, {'type': 'Date', 'value': 1668038400000}, {'type': 'Date', 'value': 1662768000000}, {'type': 'Date', 'value': 1657411200000}, {'type': 'Date', 'value': 1652140800000}, {'type': 'Date', 'value': 1646870400000}, {'type': 'Date', 'value': 1641772800000}, {'type': 'Date', 'value': 1636502400000}, {'type': 'Date', 'value': 1631232000000}, {'type': 'Date', 'value': 1625875200000}, {'type': 'Date', 'value': 1620604800000}, {'type': 'Date', 'value': 1615334400000}, {'type': 'Date', 'value': 1610236

In [36]:
sortedTimeList = timeListDate.sort()

In [55]:
def precipitation(monthlySum):
    startTime = ee.Date(monthlySum).advance(ee.Number.parse(timestep).multiply(-1), 'month')
    endTime = ee.Date(monthlySum)
    filteredChirps = chirp.filterDate(startTime, endTime)
    clippedChirps = filteredChirps.map(lambda clip: clip.clip(studyArea))
    imageAmount = clippedChirps.size()
    summedCollection = clippedChirps.sum().set({
        "Used_Images": imageAmount,
        'Start_Date': ee.Date(filteredChirps.first().get('system:time_start')),
        'End_Date': ee.Date(filteredChirps.limit(1, 'system:time_end', False).first().get('system:time_end')),
        'system:time_start': filteredChirps.first().get('system:time_start'),
        'system:time_end': ee.Date(filteredChirps.limit(1, 'system:time_end', False).first().get('system:time_end'))
    })
    time = ee.Date(summedCollection.get('system:time_end')).difference(ee.Date(summedCollection.get('system:time_start')), 'month').round()
    summedImage = summedCollection.set({
        'Observed_Months': time
    })
    return ee.Algorithms.If(
        time.gte(ee.Number.parse(timestep)),  
        summedImage,)

In [56]:
# calculate summed Chirps
precipitationSum = ee.ImageCollection.fromImages(timeListDate.map(precipitation))

In [60]:
#copy properties 
summedChirpsCollection = ee.ImageCollection(precipitationSum.copyProperties(chirp))

In [ ]:
# SPI for 12 or more months
def SpiGreaterEqual12():
    # Calculate global statistics across the entire collection
    stats = summedChirpsCollection.reduce(
        ee.Reducer.stdDev().combine(ee.Reducer.mean(), None, True)
    )
    
    # Apply SPI calculation to each image
    def calcSPI(image):
        # Make sure we have the precipitation band
        precipBand = image.select('precipitation')
        
        # Calculate SPI using the global statistics
        spi = precipBand.subtract(stats.select('precipitation_mean'))\
                         .divide(stats.select('precipitation_stdDev'))\
                         .rename('SPI')
        
        # Add the SPI band to the original image
        return image.addBands(spi)
    
    # Apply SPI calculation to each image in the collection
    return summedChirpsCollection.map(calcSPI)



In [117]:
# SPI for less than 12 months (using DOY-based statistics)
def SpiSmaller12():
    # First, calculate statistics for each day of year
    doyStats = ee.ImageCollection.fromImages(
        ee.List.sequence(1, 366).map(function(doy) {
            # Filter to this day of year across all years
            doyCollection = summedChirpsCollection.filter(
                ee.Filter.calendarRange(doy, doy, 'day_of_year')
            )
            
            # Calculate statistics for this DOY
            stats = doyCollection.select('precipitation').reduce(
                ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True)
            )
            
            # Add DOY property for later reference
            return stats.set('doy', doy)
        })
    )
    
    # Function to calculate SPI for each image
    def calcSPI(image):
        # Get the DOY for this image
        doy = ee.Date(image.get('system:time_start')).getRelative('day', 'year')
        
        # Get the statistics for this DOY
        doyStatsImage = doyStats.filter(ee.Filter.eq('doy', doy)).first()
        
        # Calculate SPI
        precipBand = image.select('precipitation')
        spi = precipBand.subtract(doyStatsImage.select('precipitation_mean'))\
                        .divide(doyStatsImage.select('precipitation_stdDev'))\
                        .rename('SPI')
        
        # Add SPI band to original image
        return image.addBands(spi)
    
    # Apply SPI calculation to each image
    return summedChirpsCollection.map(calcSPI)

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3369408953.py, line 7)

In [104]:
# If the SPI should be calculated for less than 12 months, the DOY information has to be used to find the correct images.
def SpiSmaller12():
    # calculate statistics
    def chirpStat(toStats):
        startDoY = ee.Date(toStats.get('system:time_start')).getRelative('day', 'year')
        endDoY = ee.Date(toStats.get('system:time_end')).getRelative('day', 'year')
        collectionForStats = (
            summedChirpsCollection.filter(ee.Filter.calendarRange(startDoY, 
                                                                  endDoY, 
                                                                  'day_of_year')
                                         )
        )
        return toStats.addBands(collectionForStats)
    
    stats = summedChirpsCollection.map(chirpStat)
    
    # Calculate SPI
    def calcSPI(toSPI):
        bandForSPI = toSPI.select(['precipitation'], ['SPI'])
        calc = (
            toSPI.expression('(precipitation - mean) / stdDev',
                             {
                                 'precipitation': bandForSPI,
                                 'mean': toSPI.select('precipitation_mean'),
                                 'stdDev': toSPI.select('precipitation_stdDev')})
        )
        return toSPI.addBands(calc)
    
    SPI1_ll = stats.map(calcSPI)
    return SPI1_ll

In [114]:
# SPI for 12 or more months
def SpiGreaterEqual12():
    # Calculate Statistics
    def stat(toStats):
        collectionForStats = (
            summedChirpsCollection
            .reduce(ee.Reducer.stdDev().combine(ee.Reducer.mean()))
        )
        return toStats.addBands(collectionForStats)
    stats = summedChirpsCollection.map(stat)
    
    # Calculate SPI
    def SPI(toSPI):
        bandForSPI = toSPI.select(['precipitation'], ['SPI'])
        calc = (
            toSPI.expression('(precipitation - mean) / stdDev',
                             {
                                 'precipitation': bandForSPI,
                                 'mean': toSPI.select('precipitation_mean'),
                                 'stdDev': toSPI.select('precipitation_stdDev')})
        )
        return toSPI.addBands(calc)
    SPI12_n = stats.map(SPI)
    return SPI12_n

In [115]:
SPI = ee.ImageCollection(ee.Algorithms.If(ee.Number.parse(timestep).gte(thresholdMonths), 
                                          SpiGreaterEqual12(), 
                                          SpiSmaller12()))

In [116]:
SPI.size().getInfo()

EEException: Element.get: Parameter 'object' is required and may not be null.

In [ ]:
# Define the visualization parameters for the SPI layer
SPI_monthly_vis = {
    "opacity": 1,
    "bands": ["SPI"],
    "min": -1,
    "max": 0.2,
    "palette": ['d53e4f', 'fc8d59', 'fee08b', 'ffffbf', 'e6f598', '99d594', '3288bd']
}


# Get the most recent SPI image
latest_spi = SPI.limit(1, 'system:time_start', False)

# Get the start date for the layer name
start_date = ee.Date(latest_spi.first().get('system:time_start')).format('YYYY-MM').getInfo()

# Add the SPI layer to the map
m.addLayer(
    latest_spi, 
    SPI_monthly_vis, 
    'SPI-' + str(timestep) + ' from ' + start_date
)

# Display the map
m

In [95]:
m

Map(bottom=31629.0, center=[7.879706021125551, 10.947354891686972], controls=(WidgetControl(options=['position…